In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
from importlib import reload

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from causalnex.structure.notears import from_pandas
from causalnex.structure import StructureModel
from causalnex.network import BayesianNetwork
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE
from causalnex.discretiser import Discretiser
from causalnex.discretiser.discretiser_strategy import ( DecisionTreeSupervisedDiscretiserMethod )
from causalnex.evaluation import classification_report,roc_auc
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

In [2]:
final_cols=['compactness_worst','compactness_mean','compactness_se','concavity_se','concavity_worst',
            'fractal_dimension_worst','radius_se','smoothness_se','symmetry_worst','concave points_mean',
            'smoothness_mean','smoothness_worst','symmetry_mean','fractal_dimension_mean','radius_mean',
            'radius_worst','symmetry_se','texture_se']

In [3]:
sys.path.insert(0,'C:\\Users\\Faith Bagire\\PycharmProjects\\pythonProject\\causal_impact\\modules')

In [4]:
import read_data

In [5]:
reload(read_data)

<module 'read_data' from 'C:\\Users\\Faith Bagire\\PycharmProjects\\pythonProject\\causal_impact\\modules\\read_data.py'>

In [13]:
data=read_data.read_data('../Data/data.csv')

In [14]:
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [16]:
data=data[final_cols+['diagnosis']]

In [18]:
data.head()

,compactness_worst,compactness_mean,compactness_se,concavity_se,concavity_worst,fractal_dimension_worst,radius_se,smoothness_se,symmetry_worst,concave points_mean,smoothness_mean,smoothness_worst,symmetry_mean,fractal_dimension_mean,radius_mean,radius_worst,symmetry_se,texture_se,diagnosis
0,0.6656,0.27760,0.04904,0.05373,0.7119,0.11890,1.0950,0.006399,0.4601,0.14710,0.11840,0.1622,0.2419,0.07871,17.99,25.38,0.03003,0.9053,1
1,0.1866,0.07864,0.01308,0.01860,0.2416,0.08902,0.5435,0.005225,0.2750,0.07017,0.08474,0.1238,0.1812,0.05667,20.57,24.99,0.01389,0.7339,1
2,0.4245,0.15990,0.04006,0.03832,0.4504,0.08758,0.7456,0.006150,0.3613,0.12790,0.10960,0.1444,0.2069,0.05999,19.69,23.57,0.02250,0.7869,1
3,0.8663,0.28390,0.07458,0.05661,0.6869,0.17300,0.4956,0.009110,0.6638,0.10520,0.14250,0.2098,0.2597,0.09744,11.42,14.91,0.05963,1.1560,1
4,0.2050,0.13280,0.02461,0.05688,0.4000,0.07678,0.7572,0.011490,0.2364,0.10430,0.10030,0.1374,0.1809,0.05883,20.29,22.54,0.01756,0.7813,1


In [19]:
x=data[final_cols]
y=data['diagnosis']

In [10]:
sm=StructureModel()